In [116]:
from time import sleep
from collections import deque
from copy import deepcopy
from plotly import graph_objects as go
import ipywidgets as widgets
from IPython.display import display

#kje se razgleduva tabla 6x7
plus_inf = float('inf')
minus_inf = float('-inf')

start = (
    ('.', '.', '.','.','.','.','.'),
    ('.', '.', '.','.','.','.','.'),
    ('.', '.', '.','.','.','.','.'),
    ('.', '.', '.','.','.','.','.'),
    ('.', '.', '.','.','.','.','.'),
    ('.', '.', '.','.','.','.','.'),
)

def turn(player):
    if player=='MAX':
        return 'MIN'
    else:
        return 'MAX'
        
def check_score(state):
    for row in range(6):
        for col in range(7):
            if (col+3)<7:
                if state[row][col]==state[row][col+1]==state[row][col+2]==state[row][col+3] and state[row][col]!='.': #dali 4 ima po koloni
                    return 1 if state[row][col] == 'x' else -1
    for col in range(7):
        for row in range(6):
            if (row+3)<6:
                if state[row][col]==state[row+1][col]==state[row+2][col]==state[row+3][col] and state[row][col]!='.': #dal 4 ima po redici
                    return 1 if state[row][col] == 'x' else -1
    for row in range(6):
        for col in range(6):
            if (row+3)<6 and (col+3)<6:
                if state[row][col]==state[row+1][col+1]==state[row+2][col+2]==state[row+3][col+3] and state[row][col]!='.': #glavna dijagonala
                    return 1 if state[row][col] == 'x' else -1
    for col in reversed(range(7)):
        for row in range(5):
            if (row+3)<6 and (col-3)>0:
                if state[row][col]==state[row+1][col-1]==state[row+2][col-2]==state[row+3][col-3] and state[row][col]!='.': #sporedna dijagonala
                    return 1 if state[row][col] == 'x' else -1
    return 'keep playing'

In [167]:
state = (
    ('.', '.', '.','.','.','.','.'),
    ('.', '.', '.','.','.','.','.'),
    ('.', '.', '.','.','.','o','.'),
    ('.', '.', 'o','.','x','x','.'),
    ('.', '.', 'o','x','o','x','.'),
    ('.', 'x', 'o','x','x','x','.'),
)
check_score(state)

'keep playing'

In [168]:
def tuple_to_list(state):
    return list(list(row) for row in state)

def expand_state(state, player):
    states = []
    symbol = 'x' if player == 'MAX' else 'o'
    for col_i in range(7):
        row_i = 5
        while state[row_i][col_i] != '.' and row_i >= 0:
            row_i -= 1
        if state[0][col_i] == '.':
            state_pom = tuple_to_list(state)
            state_pom[row_i][col_i] = symbol
            states.append((list(state_pom), (row_i, col_i)))
    return states

In [169]:
expand_state(state, 'MAX')

[([['.', '.', '.', '.', '.', '.', '.'],
   ['.', '.', '.', '.', '.', '.', '.'],
   ['.', '.', '.', '.', '.', 'o', '.'],
   ['.', '.', 'o', '.', 'x', 'x', '.'],
   ['.', '.', 'o', 'x', 'o', 'x', '.'],
   ['x', 'x', 'o', 'x', 'x', 'x', '.']],
  (5, 0)),
 ([['.', '.', '.', '.', '.', '.', '.'],
   ['.', '.', '.', '.', '.', '.', '.'],
   ['.', '.', '.', '.', '.', 'o', '.'],
   ['.', '.', 'o', '.', 'x', 'x', '.'],
   ['.', 'x', 'o', 'x', 'o', 'x', '.'],
   ['.', 'x', 'o', 'x', 'x', 'x', '.']],
  (4, 1)),
 ([['.', '.', '.', '.', '.', '.', '.'],
   ['.', '.', '.', '.', '.', '.', '.'],
   ['.', '.', 'x', '.', '.', 'o', '.'],
   ['.', '.', 'o', '.', 'x', 'x', '.'],
   ['.', '.', 'o', 'x', 'o', 'x', '.'],
   ['.', 'x', 'o', 'x', 'x', 'x', '.']],
  (2, 2)),
 ([['.', '.', '.', '.', '.', '.', '.'],
   ['.', '.', '.', '.', '.', '.', '.'],
   ['.', '.', '.', '.', '.', 'o', '.'],
   ['.', '.', 'o', 'x', 'x', 'x', '.'],
   ['.', '.', 'o', 'x', 'o', 'x', '.'],
   ['.', 'x', 'o', 'x', 'x', 'x', '.']],
  (

In [170]:
#pobednik kje bide onoj shto ima povekje 3ki redosledno, toest znaci mu falelo 1 za da pobedi
def evaluate_state(state):
    counter_x=0
    counter_o=0
    for row in range(6):
        for col in range(5):            
            if state[row][col]==state[row][col+1]==state[row][col+2] and state[row][col]!='.': #dali 3 ima po koloni
                if state[row][col]=='x':
                    counter_x+=1
                else: counter_o+=1
    for col in range(7):
        for row in range(4):
            if state[row][col]==state[row+1][col]==state[row+2][col] and state[row][col]!='.': #dali 3 ima po redici
                if state[row][col]=='x':
                    counter_x+=1
                else: counter_o+=1
    for row in range(4):
        for col in range(5):
            if state[row][col]==state[row+1][col+1]==state[row+2][col+2] and state[row][col]!='.': #glavna dijagonala
                if state[row][col]=='x':
                    counter_x+=1
                else: counter_o+=1
    for col in reversed(range(7)):
        for row in range(5):
            if (row+2)<5 and (col-2)>0:
                if state[row][col]==state[row+1][col-1]==state[row+2][col-2] and state[row][col]!='.': #sporedna dijagonala
                    if state[row][col]=='x':
                        counter_x+=1
                    else: counter_o+=1
    # x e max, o e min
    return (counter_x - counter_o)/50 # ako e pozitiven broj, x vodi, ako e negativen, o vodi

In [171]:
evaluate_state(state)

0.02

In [179]:
def minimax(state, player, symbol, alpha=-2, beta=2, depth=0):
    winner = check_score(state)
    if winner != 'keep playing':
        return winner, None
    # пример како да се прекине со пребарување после длабочина 3
    if depth == 3:
        return evaluate_state(state), None
    best_value = 2 if player == 'MIN' else -2
    best_move = None
    for child, move in expand_state(state, player):
        other_player = 'MIN' if player == 'MAX' else 'MAX'
        other_symbol = 'x' if symbol == 'o' else 'o'
        result, _ = minimax(child, other_player, other_symbol, alpha, beta, depth+1)
        if player == 'MIN':
            if result <= alpha:
                return result, best_move
            if result < beta:
                beta = result
            if result < best_value:
                best_value = result
                best_move = move
        elif player == 'MAX':
            if result >= beta:
                return result, best_move
            if result > alpha:
                alpha = result
            if result > best_value:
                best_value = result
                best_move = move
    return best_value, best_move


In [180]:
state

(('.', '.', '.', '.', '.', '.', '.'),
 ('.', '.', '.', '.', '.', '.', '.'),
 ('.', '.', '.', '.', '.', 'o', '.'),
 ('.', '.', 'o', '.', 'x', 'x', '.'),
 ('.', '.', 'o', 'x', 'o', 'x', '.'),
 ('.', 'x', 'o', 'x', 'x', 'x', '.'))

In [182]:
minimax(state, 'MIN', 'o', alpha=minus_inf, beta=plus_inf, depth=0)

(-1, (2, 2))